# Minimal Agentic Repo Runner Test
Test any codebase with a single click.

In [ ]:
# 1. Install repo_runner (latest from GitHub)
# The tool handles dependency conflicts automatically
!pip install --quiet --upgrade git+https://github.com/Merrex/Repo_Runner-CLI.git

# Simple verification
import subprocess
try:
    result = subprocess.run(['repo_runner', '--version'], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✅ repo_runner installed: {result.stdout.strip()}")
    else:
        print("✅ repo_runner installed (CLI check failed, but module available)")
except:
    print("✅ repo_runner installed")

In [ ]:
# 2. Intelligent repository handling - supports both zip uploads and git URLs
import os
import subprocess
import zipfile
import shutil
# 2. Intelligent repository handling - supports both zip uploads and git URLs
import os
import subprocess
import zipfile
import shutil
from google.colab import files

# Configuration
# Configuration
test_repo_path = '/content/test_repo'
git_repo_url = None  # Set this to a git URL if you want to clone instead of upload

# Example git URLs (uncomment one to use):
# git_repo_url = 'https://github.com/username/repo-name.git'
# git_repo_url = 'https://github.com/facebook/react.git'
# git_repo_url = 'https://github.com/django/django.git'

def handle_git_repo(repo_url, target_path):
    """Intelligently handle git repository - clone if not exists, pull if exists"""
    if os.path.exists(target_path):
        print(f"📁 Repository exists at {target_path}")
        try:
            # Check if it's a git repository
            result = subprocess.run(['git', 'status'], cwd=target_path, capture_output=True, text=True)
            if result.returncode == 0:
                print("🔄 Pulling latest changes...")
                subprocess.run(['git', 'pull'], cwd=target_path, check=True)
                print("✅ Repository updated successfully")
            else:
                print("⚠️ Directory exists but not a git repo, removing and cloning...")
                shutil.rmtree(target_path)
                subprocess.run(['git', 'clone', repo_url, target_path], check=True)
                print("✅ Repository cloned successfully")
        except subprocess.CalledProcessError as e:
            print(f"❌ Git operation failed: {e}")
            return False
    else:
        print(f"📥 Cloning repository from {repo_url}...")
        try:
            subprocess.run(['git', 'clone', repo_url, target_path], check=True)
            print("✅ Repository cloned successfully")
        except subprocess.CalledProcessError as e:
            print(f"❌ Git clone failed: {e}")
            return False
    return True

def handle_zip_upload():
    """Handle zip file upload and extraction"""
    print("📤 Please upload your test repository zip file...")
    uploaded = files.upload()
    
    # Clean up existing directory to prevent duplicates
    if os.path.exists(test_repo_path):
        shutil.rmtree(test_repo_path)
    
    for filename in uploaded:
        if filename.endswith('.zip'):
            print(f"📦 Extracting {filename}...")
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall(test_repo_path)
            print(f"✅ Extracted to {test_repo_path}")
            return True
    
    print("❌ No zip file found in uploads")
    return False

# Handle repository based on configuration
if git_repo_url:
    print(f"🔗 Using git repository: {git_repo_url}")
    success = handle_git_repo(git_repo_url, test_repo_path)
    if not success:
        print("❌ Failed to handle git repository, falling back to zip upload")
        success = handle_zip_upload()
else:
    print("📤 Using zip file upload")
    success = handle_zip_upload()

if not success:
    raise Exception("Failed to obtain test repository")

print(f"\n📁 Repository prepared at: {test_repo_path}")
print("🤖 Agent will automatically detect the actual repo root and key files")
print("🎯 Ready to test!")

In [ ]:
# 3. Run repo_runner fully agentically
import subprocess
import time

print("🚀 Starting repo_runner in fully agentic mode...")
print(f"📂 Target repository: {test_repo_path}")
print("⏱️ This may take a few minutes for complex repositories...")
print("🤖 Agent will automatically:")
print("   • Find the actual repository root")
print("   • Detect all services and dependencies")
print("   • Set up the environment")
print("   • Start and orchestrate services")

start_time = time.time()

try:
    result = subprocess.run(
        ['repo_runner', 'run', test_repo_path, '--mode', 'local'], 
        capture_output=True, 
        text=True,
        timeout=600  # 10 minute timeout
    )
    
    elapsed_time = time.time() - start_time
    print(f"\n⏱️ Execution time: {elapsed_time:.1f} seconds")
    print(f"Exit code: {result.returncode}")
    
    if result.returncode == 0:
        print("✅ repo_runner completed successfully!")
    else:
        print("❌ repo_runner encountered issues")
    
    print("\n--- STDOUT (last 3000 chars) ---")
    print(result.stdout[-3000:])
    
    if result.stderr:
        print("\n--- STDERR (last 2000 chars) ---")
        print(result.stderr[-2000:])
        
except subprocess.TimeoutExpired:
    print("⏰ repo_runner timed out after 10 minutes")
except Exception as e:
    print(f"❌ Error running repo_runner: {e}")

print("\n🎯 Test completed!")